In [53]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['fair-classification']


In [54]:
print(os.listdir('../input/fair-classification'))

['test_no_income.csv', 'test_sample0.csv', 'train.csv']


In [55]:
df = pd.read_csv('../input/fair-classification/train.csv')
submission_df = pd.read_csv('../input/fair-classification/test_no_income.csv')

In [72]:
df.shape

(35988, 103)

In [59]:
submission_df.columns

Index(['Age', 'workclass', 'fnlwgt', 'education', 'education-num',
       'marital-status', 'occupation', 'relationship', 'race', 'gender',
       'capital gain', 'capital loss', 'hours per week', 'native-country'],
      dtype='object')

In [60]:
df = pd.get_dummies(df)
submission_df = pd.get_dummies(submission_df)

In [61]:
df.columns

Index(['Age', 'fnlwgt', 'education-num', 'race', 'gender', 'capital gain',
       'capital loss', 'hours per week', 'income', 'workclass_?',
       ...
       'native-country_Portugal', 'native-country_Puerto-Rico',
       'native-country_Scotland', 'native-country_South',
       'native-country_Taiwan', 'native-country_Thailand',
       'native-country_Trinadad&Tobago', 'native-country_United-States',
       'native-country_Vietnam', 'native-country_Yugoslavia'],
      dtype='object', length=103)

In [63]:
missing_cols = set(df.columns) - set(submission_df.columns)

for c in missing_cols:
    submission_df[c] = 0
    
# Ensure Ordering matches
submission_df = submission_df[df.columns].drop('income', axis=1)

In [66]:
submission_df.columns

Index(['Age', 'fnlwgt', 'education-num', 'race', 'gender', 'capital gain',
       'capital loss', 'hours per week', 'workclass_?',
       'workclass_Federal-gov',
       ...
       'native-country_Portugal', 'native-country_Puerto-Rico',
       'native-country_Scotland', 'native-country_South',
       'native-country_Taiwan', 'native-country_Thailand',
       'native-country_Trinadad&Tobago', 'native-country_United-States',
       'native-country_Vietnam', 'native-country_Yugoslavia'],
      dtype='object', length=102)

In [67]:
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df, test_size=0.25)

In [68]:
X_train, y_train = train_df.drop('income', axis=1), train_df['income']
X_test, y_test = test_df.drop('income', axis=1), test_df['income']

In [69]:
from sklearn.ensemble import RandomForestClassifier

# Training
rf = RandomForestClassifier(n_estimators=1000, random_state=42, verbose=1)
rf.fit(X_train, y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:   42.1s finished


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=None,
            oob_score=False, random_state=42, verbose=1, warm_start=False)

In [70]:
# Make Predictions
predictions = rf.predict(X_test)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    2.5s finished


In [71]:
# Determine absolute error
errors = abs(predictions - y_test)

# Determine number of errors
num_correct = errors.value_counts()[0]
num_incorrect = errors.value_counts()[1]


# Determine accuracy
accuracy = (1 - (num_incorrect / num_correct)) * 100

print('Accuracy: ', round(accuracy, 2), '%.')

Accuracy:  83.05 %.


In [74]:
# Determine Bias
Th = 0.1
# DDP = np.abs()

In [77]:
X_test.shape

(8997, 102)

In [82]:
A0_indices = X_test.loc[X_test['gender'] == 0].index.values
A1_indices = X_test.loc[X_test['gender'] == 1].index.values

total_num_A0 = len(A0_indices)
total_num_A1 = len(A1_indices)

N1A0 = y_test.iloc[A0_indices]
# N1A0.unique()
total_num_A0

IndexError: positional indexers are out-of-bounds

In [73]:
# Evaluate

importances = list(rf.feature_importances_)
feature_importances = [(feature, round(importance, 5)) for feature, importance in zip(X_train.columns, importances)]
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse=True)
list_of_importances = [print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances]

Variable: fnlwgt               Importance: 0.16695
Variable: Age                  Importance: 0.15346
Variable: capital gain         Importance: 0.0983
Variable: hours per week       Importance: 0.086
Variable: marital-status_Married-civ-spouse Importance: 0.06493
Variable: education-num        Importance: 0.06308
Variable: relationship_Husband Importance: 0.04403
Variable: capital loss         Importance: 0.03057
Variable: marital-status_Never-married Importance: 0.02227
Variable: occupation_Exec-managerial Importance: 0.01849
Variable: occupation_Prof-specialty Importance: 0.01451
Variable: education_Bachelors  Importance: 0.01276
Variable: gender               Importance: 0.01249
Variable: relationship_Not-in-family Importance: 0.01181
Variable: workclass_Private    Importance: 0.01031
Variable: relationship_Wife    Importance: 0.00927
Variable: relationship_Own-child Importance: 0.00923
Variable: education_Masters    Importance: 0.00888
Variable: workclass_Self-emp-not-inc Importan

In [32]:
import datetime

file_name = '../output/test_sample_' + str(datetime.datetime.now()).replace(' ', '').replace(':','') + '.csv'
file_name

'../output/test_sample_2019-04-09224929.697337.csv'

In [39]:
submission = pd.Series(predictions)
submission

0       0
1       0
2       0
3       0
4       0
5       0
6       0
7       0
8       0
9       0
10      0
11      0
12      0
13      0
14      0
15      0
16      0
17      1
18      0
19      0
20      0
21      1
22      0
23      1
24      0
25      1
26      0
27      0
28      0
29      1
       ..
8967    1
8968    0
8969    0
8970    0
8971    1
8972    0
8973    0
8974    0
8975    0
8976    0
8977    0
8978    1
8979    1
8980    1
8981    0
8982    0
8983    0
8984    0
8985    1
8986    1
8987    0
8988    0
8989    1
8990    0
8991    1
8992    0
8993    0
8994    0
8995    0
8996    0
Length: 8997, dtype: int64

In [45]:
submission.to_csv(file_name, index_label='Id', header=['income'], index=True)